# Grey Clustering Analysis - Assessment of Natural Disaster Events in the Rio Grande do Sul Area

## 1. Disaster Data

In [35]:
import pandas as pd

In [77]:
BASE_FILE_PATH  = '../Data/Danos_Informados_{year}.xls'
AVAILABLE_YEARS = range(2013, 2024+1)
disasters_data  = pd.DataFrame()

In [78]:
for year in AVAILABLE_YEARS:
    year_data = pd.read_excel(BASE_FILE_PATH.format(year=year), skiprows=4)
    year_data['Ano'] = year

    disasters_data = pd.concat([disasters_data, year_data])


In [79]:
CATEGORIES_TO_REMOVE = ['15110 - Doenças infecciosas virais', '15120 - Doenças infecciosas bacterianas', '15210 - Infestações de animais', '15230 - Outras Infestações', '22410 - Transporte de produtos perigosos rodoviário', '23110 - Incêndios em plantas e distritos industriais, parques e depósitos', '23120 - Incêndios em aglomerados residenciais', '24100 - Colapso de edificações', '25100 - Transporte passageiros e cargas não perigosas rodoviário']
disasters_data = disasters_data.drop(disasters_data[disasters_data['COBRADE'].isin(CATEGORIES_TO_REMOVE)].index)

## 2. Clustering of Regions Regarding Natural Disaster Affects

The first thing we need to do is group the data that we have (from each municipality) into "intro-regions", as shown in the image bellow. 
![](../Docs/Regiões%20Intermediárias.png)

The data of which intro-region delimitation can be found at IBGE's web-site.

In [87]:
REGIONS_DELIMITATION = pd.read_excel('../Data/regioes_geograficas_composicao_por_municipios_2017_20180911.xls')
REGIONS_DELIMITATION = REGIONS_DELIMITATION[['nome_mun', 'nome_rgint']]
REGIONS_DELIMITATION = REGIONS_DELIMITATION[REGIONS_DELIMITATION['nome_rgint'].isin(['Porto Alegre', 'Pelotas', 'Santa Maria', 'Uruguaiana', 'Ijuí', 'Passo Fundo', 'Caxias do Sul', 'Santa Cruz do Sul - Lajeado'])]

In [88]:
#Because of Interface error between IBGE database and S2ID database, one municipality's name does not match correctly and therefore
#needs to be manually corrected
disasters_data['Município'] = disasters_data['Município'].replace(['Sant`Ana do Livramento'],  "Sant'Ana do Livramento")

In [89]:
disasters_data_by_inter_region = pd.merge(left=disasters_data, right=REGIONS_DELIMITATION, left_on='Município', right_on='nome_mun', how='left')

In [90]:
disasters_data_by_inter_region.groupby('nome_rgint').sum()

,UF,Município,Registro,Protocolo,COBRADE,Status,População,DH_Mortos,DH_Feridos,DH_Enfermos,...,"PEPL_Distribuição de combustíveis, especialmente os de uso doméstico (R$)",PEPL_Segurança pública (R$),PEPL_Ensino (R$),PEPR_Agricultura (R$),PEPR_Pecuária (R$),PEPR_Indústria (R$),PEPR_Comércio (R$),PEPR_Serviços (R$),Ano,nome_mun
nome_rgint,,,,,,,,,,,,,,,,,,,,,
Caxias do Sul,RSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRS...,Dois LajeadosPinto BandeiraBento GonçalvesMont...,06/12/201320/11/201319/11/201319/11/201318/11/...,RS-F-4306452-13214-20131119RS-F-4314548-13213-...,13214 - Tempestade Local/Convectiva - Chuvas I...,ReconhecidoReconhecidoNão reconhecidoReconheci...,5083209,10,157,3,...,3200.00,1000.00,2204661.03,2.545869e+09,4.411479e+08,1.166686e+07,609520.89,1121480.30,375725,Dois LajeadosPinto BandeiraBento GonçalvesMont...
Ijuí,RSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRS...,Cerro LargoPorto Vera CruzRedentoraTenente Por...,02/01/201525/11/201304/10/201310/10/201326/01/...,RS-F-4305207-13215-20140101RS-F-4315073-13213-...,13215 - Tempestade Local/Convectiva - Vendaval...,RegistroReconhecidoRegistroReconhecidoNão reco...,4575682,36068,50,3,...,41647.05,728.00,1964727.73,8.005153e+09,1.978162e+09,3.314988e+05,1686000.00,550430.69,733268,Cerro LargoPorto Vera CruzRedentoraTenente Por...
Passo Fundo,RSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRS...,ErnestinaSanto Antônio do PlanaltoCaseirosArvo...,27/12/201309/12/201326/12/201329/11/201325/11/...,RS-F-4307054-14110-20131227RS-F-4317756-13215-...,14110 - Estiagem13215 - Tempestade Local/Conve...,Não reconhecidoRegistroReconhecidoReconhecidoR...,6412330,4,99,67,...,151025.48,70000.00,2146754.77,1.420103e+10,2.191936e+09,1.916560e+07,2292785.00,2810000.00,1466634,ErnestinaSanto Antônio do PlanaltoCaseirosArvo...
Pelotas,RSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRS...,São Lourenço do SulCanguçuSão Lourenço do SulH...,23/12/201331/10/201305/11/201330/12/201411/12/...,RS-F-4318804-13213-20131205RS-F-4304507-12200-...,13213 - Tempestade Local/Convectiva - Granizo1...,ReconhecidoReconhecidoRegistroNão reconhecidoR...,7159292,1,0,1,...,0.00,56917.96,482160.38,4.522488e+09,1.019868e+09,1.805000e+07,988830.00,471816.68,369555,São Lourenço do SulCanguçuSão Lourenço do SulH...
Porto Alegre,RSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRS...,CristalNova HartzPalmares do SulBalneário Pinh...,27/12/201319/12/201305/12/201313/12/201316/12/...,RS-F-4306056-14110-20131227RS-F-4313060-12200-...,14110 - Estiagem12200 - Enxurradas13213 - Temp...,ReconhecidoNão reconhecidoRegistroRegistroRegi...,34730849,3349,10479,1241,...,1581844.08,2332004.41,13534590.12,2.627509e+09,4.558466e+08,3.084180e+07,46915659.50,17115940.36,769628,CristalNova HartzPalmares do SulBalneário Pinh...
Santa Cruz do Sul - Lajeado,RSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRS...,Estrela VelhaImigranteLagoãoSegredoSobradinhoV...,19/12/201316/12/201313/12/201311/12/201305/12/...,RS-F-4307815-13213-20131213RS-F-4310363-12200-...,13213 - Tempestade Local/Convectiva - Granizo1...,ReconhecidoRegistroReconhecidoRegistroRegistro...,10274666,17,284,30,...,705500.00,570250.00,4884607.82,3.689414e+09,1.113057e+09,1.959085e+08,71717050.48,12413896.65,757288,Estrela VelhaImigranteLagoãoSegredoSobradinhoV...
Santa Maria,RSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRS...,AgudoFaxinal do SoturnoRestinga SecaSão Gabrie...,23/01/201327/11/201327/11/201326/11/201312/11/...,RS-F-4300109-13215-20131231RS-F-4308003-12200-...,13215 - Tempestade Local/Convectiva - Vendaval...,RegistroReconhecidoReconhecidoReconhecidoRecon...,9501344,2,50,43,...,24000.00,2116868.00,4965033.67,5.083003e+09,7.693007e+08,3.072000e+06,6716640.00,30436985.00,559466,AgudoFaxinal do SoturnoRestinga SecaSão Gabrie...
Uruguaiana,RSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRSRS...,Sant'Ana do LivramentoManoel VianaManoel Viana...,27/12/201313/11/201325/10/201309/01/201529/12/...,RS-F-4317103-14110-20131227RS-F-4311759-12200-...,14110 - Estiagem12200 - Enxurradas13213 - Temp...,R

## 3. Clustering of Natural Disaster Season (Year)

In [39]:
disasters_data = disasters_data[['Ano', 'COBRADE', 'DH_Mortos', 'DH_Feridos', 'DH_Enfermos',
                                 'DH_Desabrigados', 'DH_Desalojados', 'DH_Desaparecidos',
                                 'DH_Outros Afetados', 'DM_Unidades Habitacionais Danificadas',
                                 'DM_Unidades Habitacionais Destruídas',
                                 'DM_Unidades Habitacionais Valor',
                                 'DA_Poluição ou contaminação da água',
                                 'DA_Poluição ou contaminação do ar',
                                 'DA_Poluição ou contaminação do solo',
                                 'DA_Diminuição ou exaurimento hídrico'
                                ]]

In [43]:
disasters_data.groupby(['Ano']).sum()

,COBRADE,DH_Mortos,DH_Feridos,DH_Enfermos,DH_Desabrigados,DH_Desalojados,DH_Desaparecidos,DH_Outros Afetados,DM_Unidades Habitacionais Danificadas,DM_Unidades Habitacionais Destruídas,DM_Unidades Habitacionais Valor,DA_Poluição ou contaminação da água,DA_Poluição ou contaminação do ar,DA_Poluição ou contaminação do solo,DA_Diminuição ou exaurimento hídrico
Ano,,,,,,,,,,,,,,,
2013,13215 - Tempestade Local/Convectiva - Vendaval...,5,115,812,12380,46429,1,531830,24734,360,8.715458e+07,0,0,0,0
2014,12200 - Enxurradas13215 - Tempestade Local/Con...,1,71,29,2516,15521,0,264969,16362,229,3.035127e+07,DE 5% A 10%,0,0,0
2015,12100 - Inundações12100 - Inundações13214 - Te...,0,24,0,2908,23701,46,351192,39584,147,9.570269e+07,DE 0% A 5%DE 5% A 10%MAIS DE 20%MAIS DE 20%DE ...,MAIS DE 20%MAIS DE 20%MAIS DE 20%,DE 0% A 5%DE 5% A 10%MAIS DE 20%MAIS DE 20%DE ...,MAIS DE 20%DE 5% A 10%DE 10% A 20%
2016,13215 - Tempestade Local/Convectiva - Vendaval...,2,346,8,1440,18321,916,243990,16636,223,7.186598e+08,DE 0% A 5%DE 5% A 10%DE 10% A 20%DE 10% A 20%D...,DE 0% A 5%,DE 0% A 5%DE 0% A 5%DE 0% A 5%DE 10% A 20%DE 1...,DE 0% A 5%DE 10% A 20%DE 0% A 5%
2017,13213 - Tempestade Local/Convectiva - Granizo1...,5,133,1,638,18171,0,907599,30764,442,7.680782e+07,MAIS DE 20%DE 10% A 20%DE 0% A 5%DE 0% A 5%MAI...,DE 0% A 5%DE 10% A 20%,DE 0% A 5%MAIS DE 20%DE 0% A 5%MAIS DE 20%DE 0...,MAIS DE 20%DE 0% A 5%MAIS DE 20%MAIS DE 20%DE ...
2018,13213 - Tempestade Local/Convectiva - Granizo1...,4,44,4,243,2111,0,353810,9449,72,2.521029e+07,DE 0% A 5%MAIS DE 20%DE 0% A 5%DE 0% A 5%MAIS ...,DE 10% A 20%DE 10% A 20%DE 10% A 20%DE 10% A 20%,DE 5% A 10%MAIS DE 20%DE 0% A 5%DE 5% A 10%MAI...,DE 5% A 10%MAIS DE 20%DE 0% A 5%DE 10% A 20%DE...
2019,14110 - Estiagem14110 - Estiagem14110 - Estiag...,1,6,1,339,3579,0,501589,15539,9,2.827599e+07,DE 0% A 5%DE 5% A 10%MAIS DE 20%DE 5% A 10%DE ...,DE 0% A 5%,DE 0% A 5%DE 5% A 10%DE 0% A 5%DE 0% A 5%DE 10...,DE 5% A 10%DE 0% A 5%MAIS DE 20%DE 5% A 10%MAI...
2020,13215 - Tempestade Local/Convectiva - Vendaval...,0,8,187,17,70,0,1250456,1187,4,1.892718e+06,MAIS DE 20%MAIS DE 20%DE 5% A 10%DE 10% A 20%D...,DE 0% A 5%DE 0% A 5%DE 10% A 20%DE 0% A 5%DE 0...,DE 0% A 5%MAIS DE 20%DE 0% A 5%DE 0% A 5%DE 0%...,MAIS DE 20%MAIS DE 20%MAIS DE 20%MAIS DE 20%MA...
2021,14110 - Estiagem14110 - Estiagem14110 - Estiag...,0,27,0,12,397,0,561791,38537,28,1.563735e+08,MAIS DE 20%DE 0% A 5%DE 0% A 5%,DE 0% A 5%,DE 10% A 20%,DE 5% A 10%DE 5% A 10%MAIS DE 20%MAIS DE 20%MA...
